# Concatenation, Analysis

In [13]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from math import ceil

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from quantum_tools import *
from experimental import *
from spam import *
from scipy.stats import gaussian_kde
from quantum_circuits import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [14]:
def evaluate_channel(channel, inputs, targets):
    d = channel.d
    kl_div = KLDiv()
    loss = kl_div(channel, inputs, targets)
    return np.real(loss)/d

## KL Divergence with Respect to Real Data

In [15]:
inputs1, targets1, _, _ = loader(data_path("belem_concatenate_2layer0"))
inputs2, targets2, _, _ = loader(data_path("belem_concatenate_2layer1"))
inputs_full, targets_full, _, _ = loader(data_path("belem_concatenate_2layer2"))

[model1, model2, model_full]=  loader(data_path("belem_concatenate_2layer_fitted.model"))

channel1 = model1.channel
channel2 = model2.channel
channel_full = model_full.channel

channel_concat = channel_to_choi_map([channel1, channel2])
channel_concat.spam = channel_full.spam

n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit1 = pqc_basic(n, 2)
U1 = circuit_to_matrix(circuit1)

circuit2 = pqc_basic(n, 2)
U2 = circuit_to_matrix(circuit2)

circuit_full = deepcopy(circuit1)
circuit_full = circuit_full.compose(circuit2)
U_full = circuit_to_matrix(circuit_full)

U_channel1 = ChoiMapStatic(U1, mode="unitary")
U_channel1.spam = channel1.spam

U_channel2 = ChoiMapStatic(U2, mode="unitary")
U_channel2.spam = channel2.spam

U_channel_full = ChoiMapStatic(U_full, mode="unitary")
U_channel_full.spam = channel_full.spam

identity_channel1 = IdentityChannel(d)
identity_channel1.spam = channel1.spam

identity_channel2 = IdentityChannel(d)
identity_channel2.spam = channel2.spam

identity_channel_full = IdentityChannel(d)
identity_channel_full.spam = channel_full.spam

In [16]:
print(f"Refrence 1 KL-Div: {evaluate_channel(identity_channel1, inputs1, targets1):.4f}")
print(f"Refrence 2 KL-Div: {evaluate_channel(identity_channel2, inputs2, targets2):.4f}")
print(f"Refrence Full KL-Div: {evaluate_channel(identity_channel_full, inputs_full, targets_full):.4f}")
print("----")
print(f"U1 KL-Div: {evaluate_channel(U_channel1, inputs1, targets1):.4f}")
print(f"U2 KL-Div: {evaluate_channel(U_channel2, inputs2, targets2):.4f}")
print(f"U_full KL-Div: {evaluate_channel(U_channel_full, inputs_full, targets_full):.4f}")
print("----")
print(f"T1 KL-Div: {evaluate_channel(channel1, inputs1, targets1):.4f}")
print(f"T2 KL-Div: {evaluate_channel(channel2, inputs2, targets2):.4f}")
print(f"T2*T1 KL-Div: {evaluate_channel(channel_concat, inputs_full, targets_full):.4f}")
print(f"T_full KL-Div: {evaluate_channel(channel_full, inputs_full, targets_full):.4f}")

Refrence 1 KL-Div: 0.1523
Refrence 2 KL-Div: 0.1389
Refrence Full KL-Div: 0.1136
----
U1 KL-Div: 0.0176
U2 KL-Div: 0.0144
U_full KL-Div: 0.0174
----
T1 KL-Div: 0.0012
T2 KL-Div: 0.0013
T2*T1 KL-Div: 0.0039
T_full KL-Div: 0.0005


In [17]:
inputs1, targets1, _, _ = loader(data_path("belem_concatenate_4layer0"))
inputs2, targets2, _, _ = loader(data_path("belem_concatenate_4layer1"))
inputs_full, targets_full, _, _ = loader(data_path("belem_concatenate_4layer2"))

[model1, model2, model_full]=  loader(data_path("belem_concatenate_4layer_fitted.model"))

channel1 = model1.channel
channel2 = model2.channel
channel_full = model_full.channel

channel_concat = channel_to_choi_map([channel1, channel2])
channel_concat.spam = channel_full.spam

n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit1 = pqc_basic(n, 4)
U1 = circuit_to_matrix(circuit1)

circuit2 = pqc_basic(n, 4)
U2 = circuit_to_matrix(circuit2)

circuit_full = deepcopy(circuit1)
circuit_full = circuit_full.compose(circuit2)
U_full = circuit_to_matrix(circuit_full)

U_channel1 = ChoiMapStatic(U1, mode="unitary")
U_channel1.spam = channel1.spam

U_channel2 = ChoiMapStatic(U2, mode="unitary")
U_channel2.spam = channel2.spam

U_channel_full = ChoiMapStatic(U_full, mode="unitary")
U_channel_full.spam = channel_full.spam


identity_channel1 = IdentityChannel(d)
identity_channel1.spam = channel1.spam

identity_channel2 = IdentityChannel(d)
identity_channel2.spam = channel2.spam

identity_channel_full = IdentityChannel(d)
identity_channel_full.spam = channel_full.spam

In [18]:
print(f"Refrence 1 KL-Div: {evaluate_channel(identity_channel1, inputs1, targets1):.4f}")
print(f"Refrence 2 KL-Div: {evaluate_channel(identity_channel2, inputs2, targets2):.4f}")
print(f"Refrence Full KL-Div: {evaluate_channel(identity_channel_full, inputs_full, targets_full):.4f}")
print("----")
print(f"U1 KL-Div: {evaluate_channel(U_channel1, inputs1, targets1):.4f}")
print(f"U2 KL-Div: {evaluate_channel(U_channel2, inputs2, targets2):.4f}")
print(f"U_full KL-Div: {evaluate_channel(U_channel_full, inputs_full, targets_full):.4f}")
print("----")
print(f"T1 KL-Div: {evaluate_channel(channel1, inputs1, targets1):.4f}")
print(f"T2 KL-Div: {evaluate_channel(channel2, inputs2, targets2):.4f}")
print(f"T2*T1 KL-Div: {evaluate_channel(channel_concat, inputs_full, targets_full):.4f}")
print(f"T_full KL-Div: {evaluate_channel(channel_full, inputs_full, targets_full):.4f}")

Refrence 1 KL-Div: 0.1263
Refrence 2 KL-Div: 0.1338
Refrence Full KL-Div: 0.1142
----
U1 KL-Div: 0.0195
U2 KL-Div: 0.0149
U_full KL-Div: 0.0214
----
T1 KL-Div: 0.0010
T2 KL-Div: 0.0008
T2*T1 KL-Div: 0.0036
T_full KL-Div: 0.0005


In [19]:
inputs1, targets1, _, _ = loader(data_path("belem_concatenate_8layer0"))
inputs2, targets2, _, _ = loader(data_path("belem_concatenate_8layer1"))
inputs_full, targets_full, _, _ = loader(data_path("belem_concatenate_8layer2"))

[model1, model2, model_full]=  loader(data_path("belem_concatenate_8layer_fitted.model"))

channel1 = model1.channel
channel2 = model2.channel
channel_full = model_full.channel

channel_concat = channel_to_choi_map([channel1, channel2])
channel_concat.spam = channel_full.spam

n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit1 = pqc_basic(n, 4)
U1 = circuit_to_matrix(circuit1)

circuit2 = pqc_basic(n, 4)
U2 = circuit_to_matrix(circuit2)

circuit_full = deepcopy(circuit1)
circuit_full = circuit_full.compose(circuit2)
U_full = circuit_to_matrix(circuit_full)

U_channel1 = ChoiMapStatic(U1, mode="unitary")
U_channel1.spam = channel1.spam

U_channel2 = ChoiMapStatic(U2, mode="unitary")
U_channel2.spam = channel2.spam

U_channel_full = ChoiMapStatic(U_full, mode="unitary")
U_channel_full.spam = channel_full.spam


identity_channel1 = IdentityChannel(d)
identity_channel1.spam = channel1.spam

identity_channel2 = IdentityChannel(d)
identity_channel2.spam = channel2.spam

identity_channel_full = IdentityChannel(d)
identity_channel_full.spam = channel_full.spam

In [20]:
print(f"Refrence 1 KL-Div: {evaluate_channel(identity_channel1, inputs1, targets1):.4f}")
print(f"Refrence 2 KL-Div: {evaluate_channel(identity_channel2, inputs2, targets2):.4f}")
print(f"Refrence Full KL-Div: {evaluate_channel(identity_channel_full, inputs_full, targets_full):.4f}")
print("----")
print(f"U1 KL-Div: {evaluate_channel(U_channel1, inputs1, targets1):.4f}")
print(f"U2 KL-Div: {evaluate_channel(U_channel2, inputs2, targets2):.4f}")
print(f"U_full KL-Div: {evaluate_channel(U_channel_full, inputs_full, targets_full):.4f}")
print("----")
print(f"T1 KL-Div: {evaluate_channel(channel1, inputs1, targets1):.4f}")
print(f"T2 KL-Div: {evaluate_channel(channel2, inputs2, targets2):.4f}")
print(f"T2*T1 KL-Div: {evaluate_channel(channel_concat, inputs_full, targets_full):.4f}")
print(f"T_full KL-Div: {evaluate_channel(channel_full, inputs_full, targets_full):.4f}")

Refrence 1 KL-Div: 0.1149
Refrence 2 KL-Div: 0.1149
Refrence Full KL-Div: 0.1205
----
U1 KL-Div: 0.0593
U2 KL-Div: 0.0637
U_full KL-Div: 0.0553
----
T1 KL-Div: 0.0004
T2 KL-Div: 0.0006
T2*T1 KL-Div: 0.0018
T_full KL-Div: 0.0002
